<a href="https://colab.research.google.com/github/iverinaivanova/complexity-factors/blob/main/embedded_clauses.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!python -m spacy download en_core_web_trf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 457.4/457.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.3/236.3 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 735.6/735.6 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 58.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import spacy

# Load the transformer-based English model
nlp = spacy.load("en_core_web_trf")

In [ ]:
def contains_adv_cl(doc):
    for token in doc:
        if token.dep_=="advcl":
            return 1
    return 0

In [ ]:
def has_complement_clause(doc):
    """
    Check if a sentence contains a complement clause of a verb or adjective.
    Complement clauses are identified by the dependency labels "ccomp" or "xcomp".
    For "xcomp", ensure it is a to-infinitival clause (not a predicative complement).
    """
    for token in doc:
        # Check if the token is a verb or adjective with a clausal complement
        if token.pos_ in {"VERB", "ADJ"}:
            for child in token.children:
                # Check for "ccomp" (finite or non-finite clausal complement)
                if child.dep_ == "ccomp":
                    return 1
                # Check for "xcomp" (to-infinitival clause)
                if child.dep_ == "xcomp" and child.tag_ == "TO":
                    return 1
    return 0

In [ ]:
def has_adnominal_clause(doc):
    """
    Check if a sentence contains a complement clause of a verb or adjective.
    Complement clauses are identified by the dependency labels "ccomp" or "xcomp".
    For "xcomp", ensure it is a to-infinitival clause (not a predicative complement).
    """
    for token in doc:
        # Check if the token is a verb or adjective with a clausal complement
        if token.pos_ == "NOUN":
            for child in token.children:
                # Check for "ccomp" (finite or non-finite clausal complement)
                if child.dep_ == "acl":
                    return 1
    return 0

In [ ]:
'''
def txt_adv_cl(doc):
    for token in doc:
      advcl = []
      if token.dep_ == "advcl":
        advcl.append(token.text)
        txt_advcl = " ".join(advcl)
        return txt_advcl

'''

In [ ]:
def process_sentences(input_file, output_file):
    """
    Process sentences from the input file and determine the presence or absence of negation markers.
    Writes the results to the output file with two columns: sentence and negation flag (0 or 1).
    """
    with open(input_file, "r", encoding="utf-8") as infile, \
         open(output_file, "w", encoding="utf-8") as outfile:

        # Write header to the output file
        outfile.write("Clause\tAdvCl\tCompCl\tAdnomCl\n")

        # Process each line in the input file
        for line in infile:
            # Strip leading/trailing whitespace
            sentence = line.strip()

            # Skip empty lines
            if not sentence:
                continue

            # Parse the sentence with spaCy
            doc = nlp(sentence)

            advcl_flag = contains_adv_cl(doc)
            compl_cl = has_complement_clause(doc)
            adnom_cl = has_adnominal_clause(doc)

            # Write the result to the output file
            outfile.write(f"{sentence}\t{advcl_flag}\t{compl_cl}\t{adnom_cl}\n")

            # Print the result to the console
            print(f"Clause: {sentence}")
            print(f"Adv Clause Flag: {advcl_flag}\n")
            print(f"CompCl: {compl_cl}\n")
            print(f"AdnomCl: {adnom_cl}\n")

    print(f"Results have been written to {output_file}")

# That-Clauses
input_file = "/finiteCCs_cleaned_final.txt"
output_file = "/embedded_clauses_that-cl.tsv"

# To-Inf
# input_file = "/all-nonfinCC-very-final-no-infm.txt"
# output_file = "/embedded-clauses_to-inf.tsv"  # Replace with your desired output file path
process_sentences(input_file, output_file)

Die letzten 5000 Zeilen der Streamingausgabe wurden abgeschnitten.

Clause: That the lobbyists have grown so persuasive.
Adv Clause Flag: 0

CompCl: 0

AdnomCl: 0

Clause: That a program like ` ` Millionaire , '' hatched in Great Britain , not in the United States , took center stage at the MIPCOM conference , the international television market held here each year where programming is bought and sold ,.
Adv Clause Flag: 0

CompCl: 0

AdnomCl: 1

Clause: That the man is a gynecologist _ a man who spends his days staring at female private parts _.
Adv Clause Flag: 0

CompCl: 0

AdnomCl: 0

Clause: That it succeeds at all _ that the film elicits moments of genuine feeling , rather than the derisive laughter it deserves _.
Adv Clause Flag: 1

CompCl: 0

AdnomCl: 0

Clause: That the president.
Adv Clause Flag: 0

CompCl: 0

AdnomCl: 0

Clause: That he is not.
Adv Clause Flag: 0

CompCl: 0

AdnomCl: 0

Clause: That they were even talking about it.
Adv Clause Flag: 0

CompCl: 0

AdnomCl: 0

